In [0]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [2]:
from google.colab import drive 
drive.mount('/mntDrive') 

Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [3]:
!mkdir data_files

import os.path
os.chdir('/content/data_files')

from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1Vy6G1IFiWdTGoHo7YaNVYw7v9Xw3XnuT',
                                    dest_path='/content/data_files/Features_Variant_1.zip',
                                    unzip=True)

mkdir: cannot create directory ‘data_files’: File exists


In [0]:
data=pd.read_csv('/content/data_files/Features_Variant_1.csv', header=None).drop_duplicates()

In [0]:
#get and fit features names
features_names=np.array(['page_likes_num',#1
               'page_checkins',#2
               'page_talking_about',#3
                'page_cat',#4
                'page_statistics',#5-29 #mean, avg etc.
                'comments_num_before_base_time',#30
                'comments_num_in_last_24_hours',#31 #last day
                'comments_num_in_last_48_to_24_hours',#32 #day before last
                'comments_num_in_first_24_hours',#33
                'comments_difference_in_last_two_days', #34 (32-31)
                'base_time', #35
                'character_num_in_post', #36
                'share_num',#37
                'post_promotion', #38 binary
                'h_local', #39 This describes the H hrs, for which we have the target variable/ comments received. 
                'post_published_weekday', #40-46 This represents the day(Sunday...Saturday) on which the post was published. 
                'base_datetime_weekday', #47-53 This represents the day(Sunday...Saturday) on selected base Date/Time. 
                'target' #54 The no of comments in next H hrs(H is given in Feature no 39).                
               ])

for index in range(5,29):
    features_names=np.insert(features_names, index, features_names[4]+'_'+str(index-4))
    
weekday=('sunday', 'monday','tuesday', 'wednesday', 'thursday', 'friday', 'saturday')    

for index in range(40,47):
    features_names=np.insert(features_names,index, features_names[39]+'_'+ weekday[index-40])
features_names=np.delete(features_names, 39)

for index in range(47,54):
    features_names=np.insert(features_names,index, features_names[46]+'_'+ weekday[index-47])
features_names=np.delete(features_names, 46)

data.columns=features_names


In [0]:
data=data.drop(columns=['post_promotion']) #post promotion has only zero values we can drop this col because it isn't informative

In [0]:
#categories:
data=data.drop(columns=['page_cat'])

data=pd.concat([data,pd.get_dummies(data['h_local'], prefix='h_local')], axis=1)
data=data.drop(columns=['h_local'])

In [0]:
#data shuffling
data=shuffle(data).reset_index(drop=True)

In [0]:
#new features:
data['comm_part_1']=data['comments_num_in_last_24_hours']/(data['comments_num_before_base_time']+1)
data['comm_part_4']=(data['comments_num_in_last_24_hours']-data['comments_num_in_last_48_to_24_hours'])/(data['comments_num_in_last_48_to_24_hours']+1)
data['comm_part_5']=(data['comments_num_in_last_24_hours']-data['comments_num_in_last_48_to_24_hours'])/(data['comments_num_before_base_time']+1)

#drop extra features:
data=data.drop(columns=[
                        'page_likes_num', 
                        'page_checkins', 
                        'page_statistics_10',
                        'page_statistics_23', 
                        'page_statistics_15',
                        'page_statistics_16',
                        'page_statistics_17',
                        'page_statistics_18',
                        'page_statistics_19',
                        'page_statistics_21',
                        'comments_num_in_first_24_hours',
       
       'character_num_in_post', 

       'post_published_weekday_sunday', 'post_published_weekday_monday',
       'post_published_weekday_tuesday', 'post_published_weekday_wednesday',
       'post_published_weekday_thursday', 'post_published_weekday_friday',
       'post_published_weekday_saturday', 

       'base_datetime_weekday_sunday',
       'base_datetime_weekday_monday', 'base_datetime_weekday_tuesday',
       'base_datetime_weekday_wednesday', 'base_datetime_weekday_thursday',
       'base_datetime_weekday_friday', 'base_datetime_weekday_saturday'])

# features transformation:
data['comments_num_in_last_48_to_24_hours']=data['comments_num_in_last_48_to_24_hours'].apply(lambda x: 0 if x==0 else 1)

In [0]:
cols=data.columns.to_list()
cols.remove("target")
data=pd.concat([data.loc[:,cols], data.loc[:,'target']], axis=1)

In [0]:
data.to_csv("prepared_facebook_data.csv",index=False)

In [0]:
cp prepared_facebook_data.csv "/mntDrive/My Drive/НИР магистратура/Оптимизация гиперпараметров алгоритмов градиентного бустинга/Практика/data/input/prepared_facebook_data.csv"